## Installation

In [1]:
!pip install semantic-version
!pip install huggingface_hub
!pip install transformers peft
!pip install unsloth
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.

## Connecting To Hugging Face and Drive

In [ ]:
from huggingface_hub import login

login(token="")

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Loading Model From Hugging Face with VLLM

In [2]:
import json
import time
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from unsloth.chat_templates import get_chat_template


# نام مدل خود در هاگینگ فیس (مدل Merge شده را انتخاب کنید)
# مثال: "username/insurance-fire-gemma-finetuned-merged"
MODEL_ID = "mahdishahsavari/gemma-3-finetune-merged"

print("⏳ در حال لود کردن مدل روی GPU...")

llm = LLM(
    model=MODEL_ID,
    dtype="bfloat16",              # برای A100 بهتر است (یا 'auto')
    max_model_len=4096,           # طول کانتکست (اگر ارور حافظه دادید، کمتر کنید)
    tensor_parallel_size=1,       # در گوگل کولب فقط ۱ (چون یک گرافیک دارید)
    gpu_memory_utilization=0.9,  # استفاده حداکثری از رم گرافیک
    trust_remote_code=True,
    enforce_eager=True            # جلوگیری از برخی کرش‌های CUDA در کولب اگر مشکل OOM گرفتیم (Out of Memory) -> از AWQ برای تبدیل به مدل کوانتایز شده 4 بیتی هم میشود استفاده کرد
)

# --- tokenizer با همون chat_template زمان فاین‌تیون ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer = get_chat_template(
    tokenizer,
    chat_template="gemma-3",   # همونی که باهاش unsloth فاین‌تیون کرده بودی
)

print("✅ مدل و توکنایزر با موفقیت لود شدند.")

/tmp/ipython-input-3470421858.py:5: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth.chat_templates import get_chat_template


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
⏳ در حال لود کردن مدل روی GPU...
INFO 12-06 11:39:14 [utils.py:253] non-default args: {'trust_remote_code': True, 'dtype': 'bfloat16', 'seed': None, 'max_model_len': 4096, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'mahdishahsavari/gemma-3-finetune-merged'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json: 0.00B [00:00, ?B/s]

WARNING 12-06 11:39:16 [arg_utils.py:1175] `seed=None` is equivalent to `seed=0` in V1 Engine. You will no longer be allowed to pass `None` in v0.13.


...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

WARNING 12-06 11:39:45 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 12-06 11:41:44 [llm.py:343] Supported tasks: ['generate']
✅ مدل و توکنایزر با موفقیت لود شدند.


## Prompt Engineering

In [ ]:
SYSTEM_PROMPT = """ """


input_data = {
    "input_block": {
        "customer_profile": {
            ""
        },
        "underwriting": {
            ""
        },
        "churn": {
            ""
        },
        "constraints": {
            ""
        }
    }
}

input_json_str = json.dumps(input_data, ensure_ascii=False)

# 👇 فرمت چت به سبک Gemma 3 / Unsloth
messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": SYSTEM_PROMPT}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "--- INPUT DATA ---\n" + input_json_str}
        ],
    },
]

final_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

## Inference

In [26]:
# پارامترهای نمونه‌برداری (Sampling Parameters)
sampling_params = SamplingParams(
    temperature=0.0,              # صفر برای منطق مطلق و خروجی JSON
    top_p=1.0,
    repetition_penalty=1.0,       # جلوگیری از تکرار (طبق درخواست شما)
    max_tokens=1024,              # فضای کافی برای جیسون خروجی
    # ignore_eos=True
    # stop=["<|im_end|>", "<|endoftext|>"]
)

## Production Run

In [27]:
import re

print("\n🚀 در حال تولید پاسخ...")
start_time = time.time()

outputs = llm.generate([final_prompt], sampling_params)

end_time = time.time()

generated = outputs[0].outputs[0]
generated = outputs[0].outputs[0]
print("Finish reason:", generated.finish_reason)
print("Num tokens:", len(generated.token_ids))
generated_text = generated.text
token_count = len(generated.token_ids)
duration = end_time - start_time

print("="*60)
print(f"⏱️ زمان سپری شده: {duration:.2f} ثانیه")
print(f"⚡ سرعت (Throughput): {token_count / duration:.2f} توکن/ثانیه")
print("="*60)

print("\n--- Raw Output ---")
print(generated_text)

import re

print("\n--- JSON Validation Check ---")
try:
    raw = generated_text

    # 1) پیدا کردن اولین {
    start = raw.find("{")
    if start == -1:
        raise ValueError("هیچ آکولاد باز { در خروجی پیدا نشد (مدل JSON را شروع نکرد).")

    # 2) پیدا کردن انتهای اولین شیء JSON با شمارش آکولادها
    depth = 0
    end = -1
    for i, ch in enumerate(raw[start:], start):
        if ch == "{":
            depth += 1
        elif ch == "}":
            depth -= 1
            if depth == 0:
                end = i
                break

    if end == -1:
        print("❌ مدل JSON را نیمه‌کاره رها کرده (هیچ آکولاد بسته کامل پیدا نشد).")
        print("بخش شروع JSON که تولید شده:")
        print(raw[start:])
        raise ValueError("JSON ناقص است و قابل parse کردن نیست.")

    # فقط اولین شیء JSON کامل را برمی‌داریم
    json_str = raw[start:end+1]

    # 3) فیکس ویرگول فارسی اگر اتفاق افتاد (اختیاری ولی می‌تونه کمک کنه)
    json_str = re.sub(r"\"،\s+\"", "\", \"", json_str)

    # 4) تلاش برای parse
    parsed_json = json.loads(json_str)

    print("✅ خروجی یک JSON معتبر است! در زیر فرمت زیبا شده را ببینید:\n")
    print(json.dumps(parsed_json, indent=4, ensure_ascii=False))

except Exception as e:
    print("❌ خطا: خروجی JSON معتبر نیست.")
    print("خام خروجی مدل:")
    print(raw)
    print(f"\nجزئیات خطا: {e}")



🚀 در حال تولید پاسخ...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Finish reason: length
Num tokens: 512
⏱️ زمان سپری شده: 37.90 ثانیه
⚡ سرعت (Throughput): 13.51 توکن/ثانیه

--- Raw Output ---
{"summary": "ریسک متوسط برای مجتمع تجاری با سازه بتنی و پوشش زلزله. پذیرش با شرایط ایمنی و افزایش جزئی حق بیمه توصیه می‌شود.", "retention_plan": [{"step": 1, "action": "الزام نصب سیستم اطفاء حریق خودکار در سالن‌ها و انبارها"}, {"step": 2, "action": "بازدید فنی از سیستم برق و تاسیسات قبل از صدور بیمه‌نامه"}, {"step": 3, "action": "اعمال فرانشیز مناسب برای زلزله و آتش‌سوزی"}], "underwriting_notes": ["طبقه خطر ۵ و فعالیت تجاری با سازه بتنی ریسک را افزایش می‌دهد، اما عدم تراکم خطر و سابقه بدونclaim مثبت است."], "cx_message_short": "ریسک متوسط برای مجتمع تجاری با سازه بتنی و پوشش زلزله. پذیرش با شرایط ایمنی و افزایش جزئی حق بیمه توصیه می‌شود."}</div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div></div